## Generate images from the <mark>SGA</mark> DCGAN Generator

In [18]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.utils as vutils
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

In [19]:
workers = 2
nc = 3
nz = 100
ngf = 64
ndf = 64
ngpu = 1

In [20]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

#### Custom dataloader class to fetch the filename of the images

In [22]:
# Define transformation
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Use ImageFolder to automatically get images and class labels
dataset = datasets.ImageFolder(root=" MUDAR PARA ADV SGA", transform=transform)

# Modify dataset to return filenames
class CustomDataset(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        image = self.loader(path)
        if self.transform:
            image = self.transform(image)
        filename = os.path.basename(path)  # Extract filename
        class_name = self.classes[target]  # Get class label from index
        return image, filename, class_name

# Create the dataloader
dataloader = DataLoader(CustomDataset(root="MUDAR PARA ADV SGA", transform=transform), batch_size=1, shuffle=False)

#### Generator class

In [23]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

#### Generate adversarial images from the dataloader source

In [24]:
netG = Generator(ngpu).to(device)
netG.load_state_dict(torch.load('Models/SGA.pth'))
netG.eval()

/tmp/ipykernel_30180/1838905899.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load('Models/Generator.pth'))


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [26]:
output_dir = "SGA"
os.makedirs(output_dir, exist_ok=True)

with torch.no_grad():
    for i, (real_img, filename, class_name) in enumerate(dataloader):
        real_img = real_img.to(device)

        # Generate a random noise vector
        noise = torch.randn(1, nz, 1, 1, device=device)

        # Generate the fake image
        fake_img = netG(noise).detach().cpu()

        # Convert to PIL image
        fake_pil = transforms.ToPILImage()(fake_img.squeeze(0))

        # Create subfolder for class if it doesn't exist
        class_dir = os.path.join(output_dir, class_name[0])
        os.makedirs(class_dir, exist_ok=True)

        # Save with the original filename
        save_path = os.path.join(class_dir, filename[0])  # filename is a list, get the first element
        fake_pil.save(save_path)

        print(f"Saved generated image as {save_path}")

Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00002701.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00003841.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00004081.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00010550.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00014050.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00018392.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00022091.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00026070.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00026421.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00026911.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00029700.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00031191.JPEG
Saved generated image as Adversarial/n02086240/ILSVRC2012_val_00036602.JPEG
Saved genera